Lab 06 Softmax Zoo_classifier-eager
 * Softmax를 사용하여 Zoo 데이터를 활용한 분류를 진행합니다.

# 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
tf.random.set_seed(777)

2.3.1


In [3]:
xy = np.loadtxt('https://raw.githubusercontent.com/deeplearningzerotoall/TensorFlow/master/tf_2.x/data-04-zoo.csv',
                delimiter=',',dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,-1]

nb_classes = 7

# Wake Y data as one hot shape
Y_one_hot = tf.one_hot(y_data.astype(np.int32),nb_classes)
print(x_data.shape,Y_one_hot.shape)

(101, 16) (101, 7)


In [4]:
# Weight and bias setting
W = tf.Variable(tf.random.normal((16,nb_classes)),name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)),name='bias')
variables = [W,b]

In [8]:
# tt.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits),dim)
def logit_fn(X):
    return tf.matmul(X,W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X,Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y,y_pred=logits,
                                                      from_logits=True)
    cost = tf.reduce_mean(cost_i)
    return cost

def grad_fn(X,Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X,Y)
        grads = tape.gradient(loss,variables)
        return grads

def prediction(X,Y):
    pred = tf.argmax(hypothesis(X),1)
    correct_prediction = tf.equal(pred,tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    return accuracy

In [9]:
def fit(X,Y,epochs=1000,verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        grads = grad_fn(X,Y)
        optimizer.apply_gradients(zip(grads,variables))
        if (i==0) | ((i+1)%verbose==0):
            #print('Loss at epochs %d : %f' %(i+1,cost_fn(X,Y).numpy()))
            acc = prediction(X,Y).numpy()
            loss = cost_fn(X,Y).numpy()
            print('Steps: {} Loss: {}, ACC: {}'.format(i+1,loss,acc))
            
fit(x_data,Y_one_hot)

Steps: 1 Loss: 2.6955952644348145, ACC: 0.40594059228897095
Steps: 100 Loss: 0.5124281048774719, ACC: 0.7920792102813721
Steps: 200 Loss: 0.3161884546279907, ACC: 0.9108911156654358
Steps: 300 Loss: 0.23416443169116974, ACC: 0.9405940771102905
Steps: 400 Loss: 0.1879921406507492, ACC: 0.9504950642585754
Steps: 500 Loss: 0.15796101093292236, ACC: 0.9603960514068604
Steps: 600 Loss: 0.13667190074920654, ACC: 0.9900990128517151
Steps: 700 Loss: 0.12070946395397186, ACC: 0.9900990128517151
Steps: 800 Loss: 0.10825730115175247, ACC: 1.0
Steps: 900 Loss: 0.09824936091899872, ACC: 1.0
Steps: 1000 Loss: 0.09001512825489044, ACC: 1.0
